In [1]:
import sqlite3

In [2]:
con = sqlite3.connect("cems.db")
cur = con.cursor()

In [20]:
cur.execute("select * from test where substr(test.op_date, -4) is cast(2001 as varchar) LIMIT 5" )
print(cur.fetchall())

[(u'WA', u'Centralia', 3845, u'BW21', u'01-01-2001', 1, 1.0, 706.0, u'', 11774.7, u'Measured', 1.64, u'Calculated', 0.359, u'Measured', 2577.369, u'Calculated', 736.6, u'Measured', 0.103, u'Calculated', 7179.3), (u'WA', u'Centralia', 3845, u'BW21', u'01-01-2001', 2, 1.0, 704.0, u'', 11156.9, u'Measured', 1.592, u'Calculated', 0.372, u'Measured', 2606.306, u'Calculated', 718.8, u'Measured', 0.103, u'Calculated', 7006.2), (u'WA', u'Centralia', 3845, u'BW21', u'01-01-2001', 3, 1.0, 702.0, u'', 10990.0, u'Measured', 1.574, u'Calculated', 0.367, u'Measured', 2561.697, u'Calculated', 716.2, u'Measured', 0.103, u'Calculated', 6980.1), (u'WA', u'Centralia', 3845, u'BW21', u'01-01-2001', 4, 1.0, 702.0, u'', 11027.6, u'Measured', 1.58, u'Calculated', 0.377, u'Measured', 2631.762, u'Calculated', 716.2, u'Measured', 0.103, u'Calculated', 6980.8), (u'WA', u'Centralia', 3845, u'BW21', u'01-01-2001', 5, 1.0, 699.0, u'', 11258.5, u'Measured', 1.601, u'Calculated', 0.381, u'Measured', 2678.697, u'Calcu

In [7]:
cur.execute("select * from test where strftime('%Y', test.op_date) = 2001 LIMIT 5" )

In [12]:
print(cur.fetchall())

[(u'01-01-2008',), (u'01-01-2008',), (u'01-01-2008',), (u'01-01-2008',), (u'01-01-2008',)]


In [22]:
cur.execute("select * from test, labels where substr(test.op_date, -4) is cast(labels.year as varchar) \
AND labels.orispl_code is test.orispl_code AND test.unitid is labels.unitid")

[(u'WA', u'Centralia', 3845, u'BW21', u'01-01-2001', 1, 1.0, 706.0, u'', 11774.7, u'Measured', 1.64, u'Calculated', 0.359, u'Measured', 2577.369, u'Calculated', 736.6, u'Measured', 0.103, u'Calculated', 7179.3, u'WA', u'Centralia', 3845, u'BW21', 2001, u'Lewis', u'WA041', 46.7559, -122.8578, u'', u'TransAlta', u'ARP : Woolley, Richard C (350) (Ended Sep 20, 2001)<br>Chambers, Linda  (2423) (Started Sep 21, 2001)', u'ARP : Enns, Lonnie S (1870)', u'Tangentially-fired', u'Coal', u'', u'')]


In [1]:
%load_ext sql

Done.


state,facility_name,orispl_code,unitid,year,county,latitude,longitude,fuel_type,fule_type_secondary


In [2]:
%%sql
sqlite:///smaller.db
drop table if exists data;
ATTACH DATABASE 'cems.db' as cems;
ATTACH DATABASE 'labels.db' as labels_db;
create table data as
select test.state as state, test.facility_name as name, test.orispl_code as orispl_code, test.unitid as unitid, op_date, year, op_hour, op_time, gload, SO2_MASS, NOX_MASS, CO2_MASS,
HEAT_INPUT as heat_input, county, latitude, longitude, fuel_type, fule_type_secondary as fuel_type_2, 
CASE WHEN (fuel_type is 'Natural Gas' or fuel_type is 'Pipeline Natural Gas' or fuel_type is 'Natural Gas, Pipeline Natural Gas')    THEN ('NG') ELSE (fuel_type) END as fuel_actual 
from cems.test as test, labels_db.labels as labels where substr(test.op_date, -4) is cast(labels.year as varchar) 
AND labels.orispl_code is test.orispl_code AND test.unitid is labels.unitid;

Done.
Done.
Done.
Done.
(sqlite3.OperationalError) no such column: facility_name [SQL: u"delete from data where state is 'TX' and (county is not 'El Paso' or facility_name is 'Montana Power Station');"]


In [4]:
%%sql
delete from data where state is 'TX' and (county is not 'El Paso' or name is 'Montana Power Station');
vacuum;

47604813 rows affected.
Done.


[]

In [6]:
% sql select * from data limit 5

Done.


state,name,orispl_code,unitid,op_date,year,op_hour,op_time,gload,SO2_MASS,NOX_MASS,CO2_MASS,heat_input,county,latitude,longitude,fuel_type,fuel_type_2,fuel_actual
WA,Centralia,3845,BW21,01-01-2001,2001,0,1.0,704.0,11493.6,2506.854,720.5,7022.0,Lewis,46.7559,-122.8578,Coal,,Coal
WA,Centralia,3845,BW21,01-01-2001,2001,1,1.0,706.0,11774.7,2577.369,736.6,7179.3,Lewis,46.7559,-122.8578,Coal,,Coal
WA,Centralia,3845,BW21,01-01-2001,2001,2,1.0,704.0,11156.9,2606.306,718.8,7006.2,Lewis,46.7559,-122.8578,Coal,,Coal
WA,Centralia,3845,BW21,01-01-2001,2001,3,1.0,702.0,10990.0,2561.697,716.2,6980.1,Lewis,46.7559,-122.8578,Coal,,Coal
WA,Centralia,3845,BW21,01-01-2001,2001,4,1.0,702.0,11027.6,2631.762,716.2,6980.8,Lewis,46.7559,-122.8578,Coal,,Coal


In [6]:
for state in ['wa', 'or', 'ca', 'id', 'nv', 'ut', 'az', 'nm', 'co', 'wy', 'mt', 'tx']:
    con = sqlite3.connect(state + ".db")
    con = sqlite3.connect("cems.db")
    cur = con.cursor()
    cur.execute("attach database '" + state + ".db' AS " + state + ";")
    cur.execute("create table " + state + ".data as \
    select state, facility_name as name, orispl_code, unitid, op_date, op_hour, op_time, gload, SO2_MASS, NOX_MASS, CO2_MASS, \
HEAT_INPUT as heat_input, county, latitude, longitude, fuel_type from data where data.state = " + UPPER('state') + ";")

OperationalError: no such column: wa